In [1]:
# $ python3 -m venv venv
# $ . ./venv/bin/activate

In [4]:
#Better
!pip install requests BeautifulSoup4 fire

In [5]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
import os, sys

import fire

In [6]:
#%%writefile ../pyscrap_url.py

def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content  #.encode(BeautifulSoup.original_encoding)
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns True if the response seems to be HTML, False otherwise.
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)
    
def get_elements(url, tag='',search={}, fname=None):
    """
    Downloads a page specified by the url parameter
    and returns a list of strings, one per tag element
    """
    
    if isinstance(url,str):
        response = simple_get(url)
    else:
        #if already it is a loaded html page
        response = url

    if response is not None:
        html = BeautifulSoup(response, 'html.parser')
        
        res = []
        if tag:    
            for li in html.select(tag):
                for name in li.text.split('\n'):
                    if len(name) > 0:
                        res.append(name.strip())
                       
                
        if search:
            soup = html            
            
            
            r = ''
            if 'find' in search.keys():
                print('findaing',search['find'])
                soup = soup.find(**search['find'])
                r = soup

                
            if 'find_all' in search.keys():
                print('findaing all of',search['find_all'])
                r = soup.find_all(**search['find_all'])
   
            if r:
                for x in list(r):
                    if len(x) > 0:
                        res.extend(x)
            
        return res

    # Raise an exception if we failed to get any data from the url
    raise Exception('Error retrieving contents at {}'.format(url))    
    
    
if get_ipython().__class__.__name__ == '__main__':
    fire(get_tag_elements)

In [40]:
res = get_elements('https://africafreak.com/100-most-influential-twitter-users-in-africa',tag='h2')
res

['100. Jeffrey Gettleman (@gettleman)',
 '99. Africa24 Media (@a24media)',
 '98. Scapegoat (@andiMakinana)',
 '97. Africa Check (@AfricaCheck)',
 '96. James Copnall (@JamesCopnall)',
 '95. Online Africa (@oafrica)',
 '94. Patrick Ngowi (@PatrickNgowi)',
 '93. DOS African Affairs (@StateAfrica)',
 '92. MoadowAJE (@Moadow)',
 '91. Brendan Boyle (@BrendanSAfrica)',
 '90. City of Tshwane (@CityTshwane)',
 '89. VISI Magazine (@VISI_Mag)',
 '88. andBeyond (@andBeyondSafari)',
 '87. This Is Africa (@ThisIsAfricaTIA)',
 '86. Sarah Carter (@sarzss)',
 '85. The EIU Africa team (@TheEIU_Africa)',
 '84. Investing In Africa (@InvestInAfrica)',
 '83. Barry Malone (@malonebarry)',
 '82. ARTsouthAFRICA (@artsouthafrica)',
 '81. Kahn Morbee (@KahnMorbee)',
 '80. Jamal Osman (@JamalMOsman)',
 '79. iamsuede™ (@iamsuede)',
 '78. Mike Stopforth (@mikestopforth)',
 '77. Equal Education (@equal_education)',
 '76. Tristan McConnell (@t_mcconnell)',
 '75. Kate Forbes (@forbeesta)',
 '74. Vanessa Raphaely (@hur

In [54]:
top_ten = res[::-1][5:15]
top_ten

['1. Trevor Noah (@Trevornoah)',
 '2. Gareth Cliff (@GarethCliff)',
 '3. Jacob G. Zuma (@SAPresident)',
 '4. News24 (@News24)',
 '5. Julius Sello Malema (@Julius_S_Malema)',
 '6. Helen Zille (@helenzille)',
 '7. mailandguardian (@mailandguardian)',
 '8. 5FM (@5FM)',
 '9. loyiso gola (@loyisogola)',
 '10. Computicket (@Computicket)']

In [58]:
twitter_handle = [i.split('(')[-1].strip(')') for i in top_ten]
twitter_handle

['@Trevornoah',
 '@GarethCliff',
 '@SAPresident',
 '@News24',
 '@Julius_S_Malema',
 '@helenzille',
 '@mailandguardian',
 '@5FM',
 '@loyisogola',
 '@Computicket']

In [27]:
url= 'https://www.atlanticcouncil.org/blogs/africasource/african-leaders-respond-to-coronavirus-on-twitter/#east-africa'
response = simple_get(url)

In [33]:
res_gov = get_elements(response, search={'find_all':{'class_':'twitter-tweet'}})
res_gov
strings=[]
for tag in res_gov:
    if tag.string !=None:
        strings.append(tag.string)



findaing all of {'class_': 'twitter-tweet'}


In [67]:
new_strings = [x for x in strings if len(x)<100]
gov_han = []
for i in range(0, len(new_strings),2):
    gov_han.append(new_strings[i])


In [35]:
gov_han

['— Eswatini Government (@EswatiniGovern1) ',
 '— Malawi Government (@MalawiGovt) ',
 '— Hage G. Geingob (@hagegeingob) ',
 '— Seychelles Ministry of Finance (@FinanceSC) ',
 '— PresidencyZA (@PresidencyZA) ',
 '— Ministry of Health Zambia (@mohzambia) ',
 '— President of Zimbabwe (@edmnangagwa) ',
 '— MinSantédj (@MinSantedj) ',
 '— Yemane G. Meskel (@hawelti) ',
 '— State House Kenya (@StateHouseKenya) ',
 '— Paul Kagame (@PaulKagame) ',
 '— Mohamed Farmaajo (@M_Farmaajo) ',
 '— South Sudan Government (@SouthSudanGov) ',
 '— Abdalla Hamdok (@SudanPMHamdok) ',
 '— TanzaniaSpokesperson (@TZSpokesperson) ',
 '— Yoweri K Museveni (@KagutaMuseveni) ',
 '— MOFA/MRE -(Angola) (@angola_Mirex) ',
 '— Amb. Willy Nyamitwe (@willynyamitwe) ',
 '— Chérif Mahamat Zene (@Cherif_MZ) ',
 '— Présidence RDC 🇨🇩 (@Presidence_RDC) ',
 '— Ali Bongo Ondimba (@PresidentABO) ',
 '— Présidence du Bénin (@PresidenceBenin) ',
 '— Roch KABORE (@rochkaborepf) ',
 '— Presidente Cabo Verde (@PresidenciaCV) ',
 '— Al

In [64]:
gov_handle = [i.split('(')[-1] for i in gov_han]
gov_handle

['@EswatiniGovern1) ',
 '@MalawiGovt) ',
 '@hagegeingob) ',
 '@FinanceSC) ',
 '@PresidencyZA) ',
 '@mohzambia) ',
 '@edmnangagwa) ',
 '@MinSantedj) ',
 '@hawelti) ',
 '@StateHouseKenya) ',
 '@PaulKagame) ',
 '@M_Farmaajo) ',
 '@SouthSudanGov) ',
 '@SudanPMHamdok) ',
 '@TZSpokesperson) ',
 '@KagutaMuseveni) ',
 '@angola_Mirex) ',
 '@willynyamitwe) ',
 '@Cherif_MZ) ',
 '@Presidence_RDC) ',
 '@PresidentABO) ',
 '@PresidenceBenin) ',
 '@rochkaborepf) ',
 '@PresidenciaCV) ',
 '@AOuattara_PRCI) ',
 '@Presidency_GMB) ',
 '@NAkufoAddo) ',
 '@President_GN) ',
 '@USEmbalo) ',
 '@PresidenceMali) ',
 '@CheikhGhazouani) ',
 '@IssoufouMhm) ',
 '@MBuhari) ',
 '@Macky_Sall) ',
 '@PresidentBio) ',
 '@MSPS_Togo) ']

In [66]:
gov_handle[:10]

['@EswatiniGovern1) ',
 '@MalawiGovt) ',
 '@hagegeingob) ',
 '@FinanceSC) ',
 '@PresidencyZA) ',
 '@mohzambia) ',
 '@edmnangagwa) ',
 '@MinSantedj) ',
 '@hawelti) ',
 '@StateHouseKenya) ']